In [2]:
import numpy as np
from scipy.special import expit as logistic_sigmoid
from scipy.optimize import minimize
from sklearn.metrics import roc_curve, auc as sklearn_auc
from scipy.stats import chi2

In [ ]:
def iso_fe(mdat):
    # Convert input to a NumPy array
    mdat = np.array(mdat)
    n, p_plus_one = mdat.shape
    p = p_plus_one - 1

    # Call sparsebasis function (assumed to be defined elsewhere)
    ans = sparsebasis(mdat, p)
    amat0 = ans['amat']
    rank = ans['rank']
    m = len(amat0) // 2
    y = ans['y']
    xmat = ans['xmat']

    # Construct amat matrix
    amat = np.zeros((m, n))
    for i in range(m):
        amat[i, int(amat0[i, 0])] = -1
        amat[i, int(amat0[i, 1])] = 1

    # Call coneproj function (assumed to be defined elsewhere)
    ans = coneproj(y, amat)
    theta = ans['theta']
    df = ans['df']

    # Return results in a dictionary
    return {
        'theta': theta,
        'df': df,
        'xmat': xmat,
        'y': y
    }

In [8]:
# XW.matrix
# Esta función crea una matriz X para todos los puntos de evaluación x0 y
# una matriz de ponderaciones W. Aplica una función kernel para calcular
# las ponderaciones en función de la distancia entre la matriz x.mat y
# cada punto de evaluación x0.mat.

def XW_matrix(x_mat, x0_mat, h, K):

    p = x_mat.shape[1]
    if len(x0_mat.shape) == 1:
        x0_mat = x0_mat.reshape(1, p)

    ngrid = x0_mat.shape[0]
    n = x_mat.shape[0]

    print(x_mat)
    print(x0_mat)
    print(ngrid)
    print(n)

    #X_mat = [x_mat - np.tile(x0_mat[i, :], (n, 1)) for i in range(ngrid)]
    X_mat = [x_mat - x0_mat.iloc[0] for i in range(ngrid)]

    print(X_mat)
    print(h)

    W_mat = 0

    if K == 'epa':
        W_mat = [kepa(np.sqrt(np.sum(X_mat[i]**2, axis=1)) / h) / h for i in range(ngrid)]
    elif K == 'gauss':
        #W_mat = [kgauss(np.sqrt(np.sum(X_mat[i]**2, axis=1)) / h) / h for i in range(ngrid)]


        # Compute the Gaussian kernel weights for each row in the DataFrame
        W_mat = [kgauss(np.sqrt((X_mat[0]**2).sum(axis=1)) / h_val) / h_val for h_val in h]

    return {'X_mat': X_mat, 'W_mat': W_mat}

# Local.lik
# Esta función calcula la probabilidad logarítmica local para la regresión
# logística. Toma un vector de coeficientes beta, una matriz X0.mat
# (que es x.mat - x0 para un punto de evaluación específico x0),
# la variable de respuesta y y el vector correspondiente de pesos w0.

def Local_lik(beta, X0_mat, y, w0):
    n, p = X0_mat.shape
    X0_mat = np.hstack((np.ones((n, 1)), X0_mat))
    u = X0_mat @ beta
    p0 = logistic_sigmoid(u)

    return -np.sum(2 * w0 * np.where(y, np.log(p0), np.log(1 - p0)))

# Local.grad
# Esta función calcula el gradiente de la función de verosimilitud
# logarítmica local. Utiliza las mismas entradas que Local.lik.

def Local_grad(beta, X0_mat, y, w0):
    n, p = X0_mat.shape
    X0_mat = np.hstack((np.ones((n, 1)), X0_mat))
    u = X0_mat @ beta
    p0 = logistic_sigmoid(u)

    return -2 * (X0_mat.T @ (np.diag(w0) @ (y - p0)))

In [ ]:
# Local.logit
#
# Esta función se ajusta a un modelo de regresión logística local en cada punto
# de evaluación x0 en x0.mat utilizando un ancho de banda especificado h y un
# kernel K.
#
#Devuelve los coeficientes estimados para cada punto de evaluación.
def Local_logit(x_mat, y, x0_mat, h, K):
    toeval = XW_matrix(x_mat, x0_mat, h, K)
    X_matrix = toeval['X_mat']  # List with all design matrices for all x0 estimation points
    W_mat = toeval['W_mat']  # Matrix with all weights vectors, for all x0 estimation points
    n_grid = x0_mat.shape[0]
    p = x_mat.shape[1]

    start = np.zeros(p + 1)
    betas = np.empty((0, p + 1))

    for i in range(n_grid):
        w0 = W_mat[i]
        X0_mat = X_matrix[i]

        f = lambda beta: Local_lik(beta, X0_mat=X0_mat, y=y, w0=w0)
        grad = lambda beta: Local_grad(beta, X0_mat=X0_mat, y=y, w0=w0)

        mle1 = minimize(f, start, jac=grad, method='BFGS')

        betas = np.vstack((betas, mle1.x))

    return betas[1:]  # Exclude the initial row of NA values

# h.CV
# Esta función realiza una validación cruzada para seleccionar el ancho de
# banda óptimo de una cuadrícula de anchos de banda candidatos h.grid.
#
# Devuelve el ancho de banda que minimiza la puntuación de validación cruzada.
def h_CV(h_grid, x_mat, y, K='gauss'):
    n_h = len(h_grid)
    n = x_mat.shape[0]

    def cv_score(h):
        betas_i = np.array([Local_logit(x_mat=np.delete(x_mat, i, axis=0), y=np.delete(y, i), x0_mat=x_mat[i, :], h=h, K=K) for i in range(n)])
        beta0_i = betas_i[:, 0]
        cv = np.sum((y - logistic_sigmoid(beta0_i))**2) / n
        return cv

    cv_values = np.array([cv_score(h) for h in h_grid])
    ind_cv = np.argmin(cv_values)
    hcv = h_grid[ind_cv]
    return hcv

In [ ]:

# logit.lik
# Esta función calcula la probabilidad logarítmica para un modelo de regresión
# logística dados los coeficientes beta, la matriz de entrada x, y la variable
# de respuesta y.
def logit_lik(beta, x, y):
    if x.ndim == 1:
        x = np.hstack(([1], x))
        u = np.dot(x, beta)
    else:
        x = np.column_stack((np.ones(x.shape[0]), x))
        u = x @ beta
    p = logistic_sigmoid(u)
    return -np.sum(np.where(y, np.log(p), np.log(1 - p)))

# logit.grad
# Esta función calcula el gradiente de la función log-verosimilitud para un
# modelo de regresión logística.
def logit_grad(beta, x, y):
    if x.ndim == 1:
        x = np.hstack(([1], x))
        u = np.dot(x, beta)
    else:
        x = np.column_stack((np.ones(x.shape[0]), x))
        u = x @ beta
    p = logistic_sigmoid(u)
    return -2 * (x.T @ (y - p))

# mon.lik
# Esta función se ajusta a un modelo de regresión logística que
# utiliza la optimización restringida (para garantizar la monotonicidad de
# los coeficientes) y devuelve los coeficientes estimados y las probabilidades.
def mon_lik(mdat):
    n, p_with_y = mdat.shape
    p = p_with_y - 1  # p = number of components of the system
    x = mdat[:, :p]
    y = mdat[:, p]

    f = lambda beta: logit_lik(beta, x=x, y=y)
    grad = lambda beta: logit_grad(beta, x=x, y=y)

    Amat = np.eye(p + 1)
    Amat[0, 0] = -1
    bvec = np.zeros(p + 2)
    start = np.array([-0.1] + [0.1] * p)

    # Adding an additional constraint for monotonicity
    Amat = np.column_stack((Amat, np.ones(p + 1)))
    bvec = np.append(bvec, 0)

    constraints = {'type': 'eq', 'fun': lambda beta: Amat.T @ beta - bvec}

    mle1 = minimize(f, start, jac=grad, method='SLSQP', constraints=constraints)
    g_beta = mle1.x

    # Predicted probability
    if x.ndim == 1:
        x = np.hstack(([1], x))
        u = np.dot(x, g_beta)
    else:
        x = np.column_stack((np.ones(x.shape[0]), x))
        u = x @ g_beta
    prob = logistic_sigmoid(u)

    return {'g_beta': g_beta, 'prob': prob}

In [ ]:
# y.pred
# Esta función predice resultados binarios en función de la probabilidad y
# un umbral alfa dado.
def y_pred(prob, alpha):
    y_0 = np.where(prob < alpha, 0, 1)
    return y_0

# roc.curve: Esta función calcula la curva ROC variando el umbral y calculando
# la sensibilidad y la especificidad.
def roc_curve_custom(y, prob):
    nc = 100
    thresholds = np.linspace(0.05, 0.95, nc)
    S_c = np.zeros(nc)
    E_c = np.zeros(nc)
    y_mat = np.empty((len(y), nc))

    for i, c in enumerate(thresholds):
        y_mat[:, i] = np.where(prob < c, 0, 1)
        y_c = y_mat[:, i]

        VN = np.sum((y_c == 0) & (y == 0))  # True Negative
        FN = np.sum((y_c == 0) & (y == 1))  # False Negative
        FP = np.sum((y_c == 1) & (y == 0))  # False Positive
        VP = np.sum((y_c == 1) & (y == 1))  # True Positive
        S_c[i] = VP / (FN + VP)  # Sensitivity
        E_c[i] = VN / (FP + VN)  # Specificity

    i_optim = np.argmin((1 - S_c)**2 + (1 - E_c)**2)
    c0 = thresholds[i_optim]
    y_c = y_mat[:, i_optim]
    return {'c0': c0, 'y_c': y_c}

# auc
# Esta función calcula el área bajo la curva ROC (AUC) comparando las
# probabilidades de verdaderos positivos y verdaderos negativos.
def auc_custom(y, prob):
    pi0 = prob[y == 0]
    pi1 = prob[y == 1]
    roc_mat = np.array([1 * (pi0_i < pi1) for pi0_i in pi0])
    return roc_mat.sum() / (len(pi0) * len(pi1))

# hoslem.gof
# Esta función realiza la prueba de bondad de ajuste de Hosmer-Lemeshow para
# modelos de regresión logística
def hoslem_gof(y, prob):
    k = len(y) // 10
    indices = np.argsort(prob)
    Ei_1 = [np.sum(prob[indices[(i * k):((i + 1) * k)]]) for i in range(10)]
    Oi_1 = [np.sum(y[indices[(i * k):((i + 1) * k)]]) for i in range(10)]
    Ei_0 = [10 - e for e in Ei_1]
    Oi_0 = [10 - o for o in Oi_1]
    HL = np.sum((np.array(Oi_1) - np.array(Ei_1))**2 / np.array(Ei_1)) + np.sum((np.array(Oi_0) - np.array(Ei_0))**2 / np.array(Ei_0))
    return chi2.sf(HL, df=8)